In [0]:
import re
import sys

import math
import numpy as np
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [0]:
#Installing dependencies for RDKit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-05-17 03:43:18--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-05-17 03:43:18--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / done

## Package Plan ##

  environment loca

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
root = "/content/drive/My Drive/"

In [0]:
import csv
def get_data(split):
    data_path =root +"QM9_Smiles_.csv"
    with open(data_path) as f:
        data = csv.reader(f)
    
        # Skip header
        next(data)
        
        # Get smiles and targets
        smiles, Y = [], []
        count=0
        for row in data:
            if (count<13000):
              smiles.append(row[0])
              Y.append(row[1:13])
              count=count+1
    
    return smiles, Y

AllSmiles, All = get_data('train')


In [0]:
import pandas as pd
import numpy as np

In [0]:
dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32'),('Col4','int32'), ('Col5','float32'), ('Col6','float32'),('Col7','int32'), ('Col8','float32'), ('Col9','float32'),('Col10','int32'), ('Col11','float32'), ('Col12','float32'),('Col13','float32'), ('Col14','float32'),('Col15','float32'), ('Col16','float32'),('Col17','float32')]
values = All
index = ['Row'+str(i) for i in range(1, len(values)+1)]

df = pd.DataFrame(values, index=index).astype(float)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,5.0509,78.30,-0.2676,0.0141,0.2818,1120.8417,0.137571,-380.797809,-380.790510,-380.789566,-380.829714,28.666
Row2,2.5256,55.40,-0.2868,-0.0597,0.2271,1180.5099,0.064339,-525.783370,-525.775913,-525.774969,-525.816245,25.955
Row3,1.5279,83.85,-0.2229,0.0807,0.3036,1029.1179,0.196525,-367.186557,-367.178749,-367.177805,-367.218256,32.345
Row4,2.1100,72.11,-0.2575,-0.0123,0.2451,1230.8475,0.136089,-455.191771,-455.182799,-455.181854,-455.226039,32.000
Row5,2.7860,75.68,-0.2791,0.0257,0.3048,1496.1036,0.168867,-440.291337,-440.280566,-440.279622,-440.327569,37.969


In [0]:
#Double check we have the right values 
s= pd.DataFrame(AllSmiles, index=index)

s.head()

,0
Row1,N#CC1NC11C2CC1C2
Row2,OC1=NON=C1OC=O
Row3,CC12CCC1CN1CC21
Row4,CN=C1OC(=O)CC1N
Row5,CC(CO)C(CO)C#N


In [0]:
len(df)

13000

In [0]:
std=df[0].std()

In [0]:
std

1.5425372727835618

In [0]:
# Normalize target values to have a mean of 0 and std of 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled=scaler.fit_transform(df)
df = pd.DataFrame(data_scaled , index=index)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,1.524938,0.378979,-1.263105,0.064793,0.650760,-0.238498,-0.327196,0.758552,0.758527,0.758527,0.758579,-0.710168
Row2,-0.112232,-2.392174,-2.140975,-1.514495,-0.506721,-0.023748,-2.537359,-2.850754,-2.850795,-2.850795,-2.850704,-1.376207
Row3,-0.759049,1.050590,0.780687,1.490003,1.112059,-0.568618,1.452052,1.097393,1.097383,1.097383,1.097421,0.193690
Row4,-0.381669,-0.370080,-0.801308,-0.500156,-0.125831,0.157421,-0.371923,-1.093429,-1.093423,-1.093423,-1.093437,0.108930
Row5,0.056587,0.061930,-1.788912,0.313028,1.137452,1.112095,0.617326,-0.722495,-0.722441,-0.722441,-0.722556,1.575396


In [0]:
def get_data(smiles_array,target_array,start, end):
   # Get smiles and targets
    smiles, Y = [], []
    for i in range(start,end):
      smiles.append(smiles_array[i])
      Y.append(target_array[i])
    
    return smiles, Y

trainSmiles, trainY = get_data(AllSmiles, df[0],2000,13000)
devSmiles, devY = get_data(AllSmiles, df[0], 1000,2000)
testSmiles, testY = get_data(AllSmiles, df[0], 0,1000)

allSmiles = trainSmiles + devSmiles + testSmiles

print(f'Num Train = {len(trainSmiles):,}')
print(f'Num Dev   = {len(devSmiles):,}')
print(f'Num Test  = {len(testSmiles):,}')

print(f'Example data point: smiles = {trainSmiles[0]}, alpha = {trainY[0]}')

Num Train = 11,000
Num Dev   = 1,000
Num Test  = 1,000
Example data point: smiles = CC1=C2NCCN2N=N1, alpha = 1.8780066871728993


In [0]:
allSmiles[0]

'CC1=C2NCCN2N=N1'

In [0]:
class QM9Dataset(Dataset):
    def __init__(self, X, Y):
      self.X, self.Y = X, Y
      assert len(X) == len(Y)

    def __len__(self):
       return len(self.X)

    def __getitem__(self, i):
      return np.array(self.X[i]), self.Y[i]

In [0]:
batch_size = 64
lr = 1e-3
weight_decay = 1e-4
max_len = 25
embedding_size = 1
hidden_size = 64
output_size = 1  
dropout = 0.6
use_cuda = True

In [0]:
def param_count(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)
  
def mae(targets, preds):
    return mean_absolute_error(targets, preds)

In [0]:
# Define vocab
vocab = {char for smiles in AllSmiles for char in smiles}

print(f'Vocab = {vocab}')

# Create word to index mapping
padding_idx = 0
char_to_index = {char: index + 1 for index, char in enumerate(vocab)}
vocab_size = len(char_to_index) + 1

print(f'Vocab size = {vocab_size:,}')

Vocab = {'(', ']', '[', '+', 'N', ')', 'O', 'F', '3', '4', 'C', 'H', '5', '-', '=', '#', '2', '1'}
Vocab size = 19


In [0]:
char_to_index 

{'#': 16,
 '(': 1,
 ')': 6,
 '+': 4,
 '-': 14,
 '1': 18,
 '2': 17,
 '3': 9,
 '4': 10,
 '5': 13,
 '=': 15,
 'C': 11,
 'F': 8,
 'H': 12,
 'N': 5,
 'O': 7,
 '[': 3,
 ']': 2}

In [0]:
biggest_mol_size = max([len(smiles) for smiles in allSmiles])
print("Total number of smiles= ", len(AllSmiles))
print("size of largest molecule = ", biggest_mol_size)

Total number of smiles=  13000
size of largest molecule =  27


In [0]:
trainX = [[char_to_index[char] for char in smiles] for smiles in trainSmiles]
devX =   [[char_to_index[char] for char in smiles] for smiles in devSmiles]
testX =  [[char_to_index[char] for char in smiles] for smiles in testSmiles]

print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')


print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')

Smiles string = CC1=C2NCCN2N=N1
Indices of first train SMILES = [11, 11, 18, 15, 11, 17, 5, 11, 11, 5, 17, 5, 15, 5, 18]
Last five indices = [17, 5, 15, 5, 18]
Smiles string = CC1OC1(CO)C1CO1
Indices of second train SMILES = [11, 11, 18, 7, 11, 18, 1, 11, 7, 6, 11, 18, 11, 7, 18]
Last five indices = [11, 18, 11, 7, 18]


In [0]:
#add all elemnets the pad with os at the end so that they have the same length
trainX = [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in trainX]
devX =   [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in devX]
testX =  [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in testX]


print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')


print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')


Smiles string = CC1=C2NCCN2N=N1
Indices of first train SMILES = [11, 11, 18, 15, 11, 17, 5, 11, 11, 5, 17, 5, 15, 5, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]
Smiles string = CC1OC1(CO)C1CO1
Indices of second train SMILES = [11, 11, 18, 7, 11, 18, 1, 11, 7, 6, 11, 18, 11, 7, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]


In [0]:
def train_epoch(model, train_loader, optimizer, epoch, std):
    model.train()  # Set the nn.Module to train mode. 
    total_loss = 0
    total_mae = 0
    num_samples = len(train_loader.dataset)
    num_batches = 0
    for batch_idx, (data, target) in enumerate(train_loader):  # 1) get batch
        # Adjust dimensions of target and cast to float
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
      
        # Reset gradient data to 0
        optimizer.zero_grad()
        
        # Get prediction for batch
        output = model(data)
        
        # 2) Compute loss
        loss = F.mse_loss(output, target)
        
        # 3) Do backprop
        loss.backward()
        
        # 4) Update model
        optimizer.step()
        
        # Do book-keeping to track rmse and avg loss
        total_loss += loss.detach()  # Don't keep computation graph 
        total_mae += mae(target.cpu().data.numpy(), output.cpu().data.numpy())
        num_batches += 1
    return ( total_loss/num_samples)

In [0]:
def eval_epoch(model, test_loader, std):
    model.eval()
    test_loss = 0
    test_mae = 0
    num_batches = 0
    for data, target in test_loader:
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
        
        test_loss += F.mse_loss(output, embed).item()  # sum up batch loss
        test_mae += mae(target.cpu().data.numpy(), output.cpu().data.numpy())
        num_batches += 1

    test_loss /= len(test_loader.dataset)
    test_mae /= num_batches
    return test_mae*std

In [0]:
# Build Dataset
train = QM9Dataset(trainX, trainY)
dev = QM9Dataset(devX, devY)
test = QM9Dataset(testX, testY)

# Build DataLoader
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout):
        super(LSTMModel, self).__init__()
        
        # Embedding layer
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)
        
        # LSTM (RNN)
        self.rnn_encoder = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        
       # LSTM (RNN)
        self.rnn_decoder = nn.LSTM(
            input_size=hidden_size,
            hidden_size=embedding_size,
            batch_first=True
        )

        
        # Fully connected layer
        self.output = nn.Linear(hidden_size, output_size)
        
        # Dropout (regularization)
      #  self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):  # batch_size x seq_length
        # Embed
        embedded = self.embed(x)  # batch_size x seq_length x embedding_size
         
        # Run RNN
        o, _ = self.rnn_encoder(embedded)  # batch_size x seq_length x hidden_size
        m, _ = self.rnn_decoder(o)  # batch_size x seq_length x hidden_size
          
        # Dropout
       # o = self.dropout(o)  # batch_size x seq_length x hidden_size
        
        # Max pooling across sequence
        n, _ = torch.max(o, dim=1)    # batch_size x hidden_size
        
        # Output layer
        out = self.output(n)  # batch_size x output_size
        
        return embedded, m,n

In [0]:
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.kernel_ridge import KernelRidge
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

"""Evaluates latent space quality via a linear regression downstream task."""
def linear_reg(X,Y,std):

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)
  params = { 'kernel':['laplacian'],'alpha':[0.001, 0.01,0.1,1,10,100]}
  #clf = KernelRidge(alpha=1.0, kernel='laplacian',)
  clf = GridSearchCV(KernelRidge(), params, cv=5)
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  mae = mean_absolute_error(y_test,y_pred)*std 
  return mae


def evaluate_embedding(embeddings, labels, std):
    xa, ya= embeddings.cuda().cpu(),labels.cuda().cpu(),
    x,y=np.array(xa.detach().numpy()),np.array(ya.detach().numpy())
    #print(x.shape, y.shape)

    linreg_accuracies = [linear_reg(x, y, std) for _ in range(1)]
    #print('LinReg', np.mean(linreg_accuracies))

    return np.mean(linreg_accuracies)

In [0]:
def train(model, train_loader, optimizer, epoch, std):
    model.train()  # Set the nn.Module to train mode. 
    total_loss = 0
    total_mae = 0
    num_samples = len(train_loader.dataset)
    num_batches = 0
    for batch_idx, (data, target) in enumerate(train_loader):  # 1) get batch
        # Adjust dimensions of target and cast to float
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
      
        # Reset gradient data to 0
        optimizer.zero_grad()
        
        # Get prediction for batch
        embed,output,n = model(data)
        
        # 2) Compute loss
        loss = F.mse_loss(output, embed)
        
        # 3) Do backprop
        loss.backward()
        
        # 4) Update model
        optimizer.step()
        
        y=evaluate_embedding(n, target, std)
        # Do book-keeping to track rmse and avg loss
        total_loss += loss.detach()  # Don't keep computation graph 
        num_batches += 1
    return (y)

In [0]:
model = LSTMModel(vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout)
print(model)
print(f'Number of parameters = {param_count(model):,}')

# Move to cuda
if use_cuda and torch.cuda.is_available():
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)
best_val_error = None
for epoch in range(1, 20):
    mae = train(model, train_loader, optimizer, epoch,std)
    print('Epoch: {}'.format(epoch))
    print('Kernel Ridge Regression Mean Absolute Error : {}'.format(mae))

   

LSTMModel(
  (embed): Embedding(19, 1, padding_idx=0)
  (rnn_encoder): LSTM(1, 64, batch_first=True)
  (rnn_decoder): LSTM(64, 1, batch_first=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)
Number of parameters = 17,504
Epoch: 1
Kernel Ridge Regression Mean Absolute Error : 1.2192138362052207
Epoch: 2
Kernel Ridge Regression Mean Absolute Error : 1.0304034572794587
Epoch: 3
Kernel Ridge Regression Mean Absolute Error : 1.1899304460831566
Epoch: 4
Kernel Ridge Regression Mean Absolute Error : 1.0866744588915118
Epoch: 5
Kernel Ridge Regression Mean Absolute Error : 1.0703442862259933
Epoch: 6
Kernel Ridge Regression Mean Absolute Error : 0.9646780925805549
Epoch: 7
Kernel Ridge Regression Mean Absolute Error : 1.4402072351106245
Epoch: 8
Kernel Ridge Regression Mean Absolute Error : 1.074954514334586
Epoch: 9
Kernel Ridge Regression Mean Absolute Error : 0.7850762808700198
Epoch: 10
Kernel Ridge Regression Mean Absolute Error : 0.8661578879164747
Epoch: 11
Kernel R